In [ ]:
import sys
sys.path.append('..')

import numpy as np
# import pandas as pd

from emonet.models import EmoNet

In [2]:
# 
def get_layer_name(x):
    for y in ['.weight', '.bias']:
        x = x.replace(y, '')
    return x

model = EmoNet(n_expression=8).to("cuda")
params_tot = 0
layers = set()
print('IDX: NAME | SHAPE | PARAMS | GRAD')
for idx, (name, p) in enumerate(model.named_parameters()):
    shape = tuple(p.shape)
    params = np.prod(shape)
    params_tot += params
    froz = 'FROZEN' if not p.requires_grad else "      "
    layer = get_layer_name(name)
    if layer in layers:
        name = ' ... ' + name.split('.')[-1]
        layer_idx = ''
    else:
        layers.add(layer)
        layer_idx = len(layers)
    print(' {:>4} | {:>4} | {:<30} | {:<18} | {:>8_} | {} | {:_}'.format(idx, layer_idx, name, str(shape), params, froz, params_tot))

IDX: NAME | SHAPE | PARAMS | GRAD
    0 |    1 | conv1.weight                   | (64, 3, 7, 7)      |    9_408 | FROZEN | 9_408
    1 |      |  ... bias                      | (64,)              |       64 | FROZEN | 9_472
    2 |    2 | bn1.weight                     | (64,)              |       64 | FROZEN | 9_536
    3 |      |  ... bias                      | (64,)              |       64 | FROZEN | 9_600
    4 |    3 | conv2.bn1.weight               | (64,)              |       64 | FROZEN | 9_664
    5 |      |  ... bias                      | (64,)              |       64 | FROZEN | 9_728
    6 |    4 | conv2.conv1.weight             | (64, 64, 3, 3)     |   36_864 | FROZEN | 46_592
    7 |    5 | conv2.bn2.weight               | (64,)              |       64 | FROZEN | 46_656
    8 |      |  ... bias                      | (64,)              |       64 | FROZEN | 46_720
    9 |    6 | conv2.conv2.weight             | (32, 64, 3, 3)     |   18_432 | FROZEN | 65_152
   10 |    7

In [3]:
# torchinfo
from torchinfo import summary
summary(model, input_size=(1, 3, 256, 256))

Layer (type:depth-idx)                   Output Shape              Param #
EmoNet                                   [1]                       --
├─Conv2d: 1-1                            [1, 64, 128, 128]         (9,472)
├─BatchNorm2d: 1-2                       [1, 64, 128, 128]         (128)
├─ConvBlock: 1-3                         [1, 128, 128, 128]        --
│    └─BatchNorm2d: 2-1                  [1, 64, 128, 128]         (128)
│    └─Conv2d: 2-2                       [1, 64, 128, 128]         (36,864)
│    └─BatchNorm2d: 2-3                  [1, 64, 128, 128]         (128)
│    └─Conv2d: 2-4                       [1, 32, 128, 128]         (18,432)
│    └─BatchNorm2d: 2-5                  [1, 32, 128, 128]         (64)
│    └─Conv2d: 2-6                       [1, 32, 128, 128]         (9,216)
│    └─Sequential: 2-7                   [1, 128, 128, 128]        --
│    │    └─BatchNorm2d: 3-1             [1, 64, 128, 128]         (128)
│    │    └─ReLU: 3-2                    [1, 64, 

In [5]:
# torchview
import os
os.environ["PATH"] += os.pathsep + r"C:\Program Files\Graphviz\bin"

from torchview import draw_graph
graph = draw_graph(model, input_size=(1, 3, 256, 256))
graph.visual_graph.render("model_structure", format="png")


(process:29620): Pango-WARNING **: 20:46:22.334: couldn't load font "Linux libertine Not-Rotated 10", falling back to "Sans Not-Rotated 10", expect ugly output.


'model_structure.png'